In [1]:
from utils import *
import numpy as np
import pickle
# import torch

In [2]:
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

In [3]:
hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb')) 
english_alphabet_to_index=pickle.load(open('vocab_tools/english_alphabet_to_index.pickle', 'rb')) 

In [4]:
X_train=np.load('simple_data/X_train.npy')
X_valid=np.load('simple_data/X_val.npy')

y_train=np.load('simple_data/y_train.npy')
y_valid=np.load('simple_data/y_val.npy')

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/anaconda/envs/shared_conda_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Eng_Hind_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, in_file, out_file, root_dir='simple_data',device='cuda'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input = torch.tensor(np.load(root_dir+'/'+in_file))
        self.output = torch.tensor(np.load(root_dir+'/'+out_file))
        
        assert(len(self.input)==len(self.output),"Error: I/O Lengths must be same")
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X=self.input[idx]
        X=X.to(device)
        y=self.output[idx]
        y=y.to(device)
        


        sample = {'input': X, 'output': y}

        return sample

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
training_data=Eng_Hind_Dataset("X_train.npy","y_train.npy",device=device)
val_data=Eng_Hind_Dataset("X_val.npy","y_val.npy",device=device)
test_data=Eng_Hind_Dataset("X_test.npy","y_test.npy",device=device)

In [10]:
train_dataloader = DataLoader(training_data, batch_size=16,shuffle=True)

In [11]:
val_dataloader = DataLoader(val_data, batch_size=16,shuffle=True)

In [12]:
import torch.nn as nn

In [13]:
def cell_type(mode:str='rnn'):
    mode=mode.lower()
    if mode == 'rnn':
        return nn.RNN
    elif mode =='gru':
        return nn.GRU
    else:
        return nn.LSTM
        

In [14]:
class Encoder(nn.Module):
    """
    Input :
        - source batch
    Layer : 
        source batch -> Embedding -> LSTM
    Output :
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    input_dim : int
        Input dimension, should equal to the source vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    def __init__(self, input_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()
        
        # create embedding layer
        self.embedding = nn.Embedding(input_size, embed_size,padding_idx=english_alphabet_to_index['.'])

        #creating LSTM/GRU/RNN cell
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi,batch_first=True)
        self.cell_mode=cell_mode
        
        

    def forward(self, input_batch: torch.LongTensor):
        """

        Parameters
        ----------
        src_batch : 2d torch.LongTensor
            Batched tokenized source sentence of shape [sent len, batch size].

        Returns
        -------
        hidden, cell : 3d torch.LongTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        embedded = self.embedding(input_batch) # [sent len, batch size, emb dim]
        #print('encoder embd',embedded.shape)
        
        if self.cell_mode.lower()=='lstm':
            outputs, (hidden, cell) = self.cell(embedded)
            
            
        else:
            outputs, hidden = self.cell(embedded)
            cell=outputs
        # outputs -> [sent len, batch size, hidden dim * n directions]
        return hidden, cell

In [15]:
class Decoder(nn.Module):
    """
    Input :
        - first token in the target batch
        - LSTM hidden state from the encoder
        - LSTM cell state from the encoder
    Layer :
        target batch -> Embedding -- 
                                   |
        encoder hidden state ------|--> LSTM -> Linear
                                   |
        encoder cell state   -------
        
    Output :
        - prediction
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    output : int
        Output dimension, should equal to the target vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    
    def __init__(self, output_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()


        self.embedding = nn.Embedding(output_size, embed_size,padding_idx=hindi_alphabet_to_index['.'])
        
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi,batch_first=True)
        self.out = nn.Linear(hid_size, output_size)
        
        self.output_size=output_size
        self.cell_mode=cell_mode
        

    def forward(self, trg: torch.LongTensor, hidden: torch.FloatTensor, cell: torch.FloatTensor):
        """

        Parameters
        ----------
        trg : 1d torch.LongTensor
            Batched tokenized source sentence of shape [batch size].
            
        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]

        Returns
        -------
        prediction : 2d torch.LongTensor
            For each token in the batch, the predicted target vobulary.
            Shape [batch size, output dim]

        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        # [1, batch size, emb dim], the 1 serves as sent len
        embedded = self.embedding(trg.unsqueeze(1))
        if self.cell_mode.lower()=='lstm':
            #print('decoder embed',embedded.shape)
            outputs, (hidden, cell) = self.cell(embedded, (hidden, cell))
        else:
            outputs, hidden = self.cell(embedded, hidden)
            cell=hidden
        prediction = self.out(outputs.squeeze(1))
        return prediction, hidden, cell

In [16]:
torch.tensor([1]).to(device)

tensor([1], device='cuda:0')

In [17]:
len(val_dataloader)

256

In [18]:
next(iter(train_dataloader))['input'].shape

torch.Size([16, 30])

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, source_batch, target_batch, teacher_forcing_ratio=0.5):

        batch_size,max_len  = target_batch.shape
        #print(max_len,batch_size)
        target_vocab_size = self.decoder.output_size
        #print(target_vocab_size)

        # tensor to store decoder's output
        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)
#         outputs = torch.zeros( batch_size,max_len, target_vocab_size).to(self.device)

        # last hidden & cell state of the encoder is used as the decoder's initial hidden state
        hidden, cell = self.encoder(source_batch)       

        trg = target_batch[:,0]
#         trg = target_batch[0]
        #print('target[0]>',trg)
        #print('tbs',target_batch.shape)
        #print('trg',trg.shape)
        for i in range(1, max_len):
            #print('i,hid,cel,',i, hidden.shape,cell.shape)
            prediction, hidden, cell = self.decoder(trg, hidden, cell)
            #print('pred',prediction.shape)
            #print('outputs',outputs.shape)
            
            
            outputs[i] = prediction
            #print('pred',prediction.shape)

            if random.random() < teacher_forcing_ratio:
                trg = target_batch[:,i]
                #print(1, trg.shape)
            else:
                trg = prediction.argmax(1)
                #print(2, trg.shape)

        return outputs


        
        

In [20]:
def train(seq2seq, iterator, optimizer, criterion):
    
    
    seq2seq.train()
    
    epoch_loss = 0.0
    correct=0
    correct_char=0
    tot_char=0
    for batch in iterator:
        optimizer.zero_grad()
        outputs = seq2seq(batch['input'], batch['output'])
        batch_label=batch['output'].transpose(0,1)
        
        #print('outputs',outputs.shape)
        #print('outputsbatchs',batch['output'].shape)
        #print('batch_label',batch_label.shape)
        _, predicted = torch.max(outputs, dim=2)
        #print('wow_preds',predicted.shape)
        
        outputs_flatten = outputs.view(-1, outputs.shape[-1])
        trg_flatten = batch_label.reshape(-1)
        
#         #print('shape output_flat',outputs_flatten.shape)
        #print('shape output_flat',outputs_flatten)
        #print('shape trg_flatten',trg_flatten)

        trg_flatten.requires_grad=False
        loss = criterion(outputs_flatten, trg_flatten)
        
        
            
        if torch.all(torch.eq(batch['output'].transpose(0,1),predicted)):
            correct+=1
        #print('shape output_flat',outputs_flatten.shape)
        

        loss.backward()
        optimizer.step()
        
        correct_char += torch.sum(predicted == batch['output'].transpose(0,1)).item()
        tot_char += batch['output'].numel()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator), correct/len(iterator),correct_char/tot_char

In [21]:


def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    with torch.no_grad():
        for batch in iterator:
            outputs = seq2seq(batch['input'], batch['output'],teacher_forcing_ratio=0)
            batch_label=batch['output'].transpose(0,1)

            #print('outputs',outputs.shape)
            #print('outputsbatchs',batch['output'].shape)
            #print('batch_label',batch_label.shape)
            _, predicted = torch.max(outputs, dim=2)
            #print('wow_preds',predicted.shape)

            outputs_flatten = outputs.view(-1, outputs.shape[-1])
            trg_flatten = batch_label.reshape(-1)

            loss = criterion(outputs_flatten, trg_flatten)


            if torch.all(torch.eq(batch['output'].transpose(0,1),predicted)):
                correct+=1


#             epoch_loss += loss.item()
#         for batch in iterator:
#             # turn off teacher forcing
#             outputs = seq2seq(batch['input'], batch['output'], teacher_forcing_ratio=0)
#             _, predicted = torch.max(outputs, dim=2)
            

#             # trg = [trg sent len, batch size]
#             # output = [trg sent len, batch size, output dim]
#             outputs_flatten = outputs.view(-1, outputs.shape[-1])
#             trg_flatten = batch['output'].view(-1)
#             loss = criterion(outputs_flatten, trg_flatten)
            
#             for jik in range(len(batch['output'])):
#                 if torch.all(torch.eq(batch['output'][jik],predicted)):
#                     correct+=1
                    
            correct_char += torch.sum(predicted == batch['output'].transpose(0,1)).item()
            tot_char += batch['output'].numel()
        
            
            epoch_loss += loss.item()

    return epoch_loss / len(iterator), correct/len(iterator),correct_char/tot_char



In [22]:
def epoch_time(start_time, end_time):
    e_time = end_time - start_time
    mins = e_time // 60
    secs = e_time%60
    return mins, secs,

In [23]:
def count_params(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [24]:
import time
import random

In [25]:
import torch.optim as optim
def make_model(train_iterator,valid_iterator,N_EPOCHS=20):
    E=Encoder(30,128,128,2,'lstm',0.2,False)
    E=E.to(device)
    D=Decoder(68,128,128,2,'lstm',0.2,False)
    D=D.to(device)
    S=Seq2Seq(E,D,device)
    S.to(device)    
    print(f'The model has {count_params(S):,} trainable parameters')
    
    optimizer = optim.Adam(S.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=hindi_alphabet_to_index['.'])
    criterion=criterion.to(device)
    
    best_valid_loss = float('inf')
#     return S

    for epoch in range(N_EPOCHS):    
        start_time = time.time()
        train_loss,train_acc,train_stuff = train(S, train_iterator, optimizer, criterion)
        valid_loss,valid_acc,val_stuff = evaluate(S, valid_iterator, criterion)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(S.state_dict(), 'model1.pt')

        # it's easier to see a change in perplexity between epoch as it's an exponential
        # of the loss, hence the scale of the measure is much bigger
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        print(f'\t Relaxed Train. Acc: {train_stuff*100:.2f}% | Relaxed Val. Acc: {val_stuff*100:.2f}%')
        
    return S

    

In [26]:
train_iterator=train_dataloader
valid_iterator=val_dataloader
SS=make_model(train_iterator,valid_iterator,N_EPOCHS=10)

The model has 549,700 trainable parameters
Epoch: 01 | Time: 1.0m 17.69s
	 Train Loss: 2.709 | Train Acc: 0.00%
	 Val. Loss: 2.063 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 12.70% | Relaxed Val. Acc: 15.55%
Epoch: 02 | Time: 0.0m 58.00s
	 Train Loss: 1.653 | Train Acc: 0.00%
	 Val. Loss: 1.668 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 20.62% | Relaxed Val. Acc: 18.35%
Epoch: 03 | Time: 0.0m 30.19s
	 Train Loss: 1.396 | Train Acc: 0.00%
	 Val. Loss: 1.574 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 22.95% | Relaxed Val. Acc: 19.35%
Epoch: 04 | Time: 0.0m 30.19s
	 Train Loss: 1.281 | Train Acc: 0.00%
	 Val. Loss: 1.522 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 24.04% | Relaxed Val. Acc: 19.77%
Epoch: 05 | Time: 0.0m 30.19s
	 Train Loss: 1.218 | Train Acc: 0.00%
	 Val. Loss: 1.496 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 24.66% | Relaxed Val. Acc: 20.07%
Epoch: 06 | Time: 0.0m 30.20s
	 Train Loss: 1.171 | Train Acc: 0.00%
	 Val. Loss: 1.471 |  Val. Acc: 0.00%
	 Relaxed Train. Acc: 25.14% | Rela

In [27]:
example=torcaaah.randn(16,30)

NameError: name 'torcaaah' is not defined

In [ ]:
example.view(-1)

In [ ]:
example.transpose(1,0).shape

In [ ]:
a=torch.tensor([[[1,0],
                 [2,30]],[[6,7],
                 [8,9]]])

In [ ]:
a.shape

In [ ]:
a.transpose(0,1)

In [ ]:
a.transpose(1,0)

In [ ]:
aa=torch.randn(30,16)

In [ ]:
bb=aa.transpose(0,1)

In [ ]:
bb.shape

In [ ]:
aa.shape

In [ ]:
bb.reshape(-1)==aa.view(-1)

In [ ]:
a.view(-1)

In [ ]:

# # ignore the pcriterion = nn.CrossEntropyLoss(adding index when calculating the loss
# PAD_IDX = target.vocab.stoi['<pad>']
# criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)



In [28]:
a=next(iter(train_dataloader))

In [29]:
bya=SS(a['input'],a['output'])

In [30]:
a['output'].shape

torch.Size([16, 30])

In [31]:
bya.shape

torch.Size([30, 16, 68])

In [32]:
import math

In [33]:
def word_from_torchies(torchie1,index_toalp):
    torchie=torchie1.cpu().numpy()
    return word_from_vecs(torchie,index_toalp,False)

In [34]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [39]:
word_from_torchies(bya.argmax(2)[3],index_to_hindi_alphabet)

Invalid Word


In [40]:
index_to_hindi_alphabet[65]

'ङ'

In [41]:
def make_wrd(stuff):
    asa=[]
    for k in stuff.cpu().numpy():
        asa.append(index_to_hindi_alphabet[k])
    return "".join(asa)
    

In [42]:
make_wrd(bya.transpose(0,1).argmax(2)[10])

'<नंदािषारिकर्तायय>>>>>>>>>>>>>'

In [43]:
bya.transpose(0,1).argmax(2)[10]

tensor([ 0,  5, 13,  7,  9,  4, 48,  9, 11,  4, 10, 11,  6, 14,  9,  8,  8,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1], device='cuda:0')

In [44]:
for i in range(16):
    print('................')
    print(i)
    print(word_from_torchies(a['output'][i],index_to_hindi_alphabet),\
         '---',\
         make_wrd(bya.transpose(0,1).argmax(2)[i])\
         )


................
0
कंटैक्टलेस --- <कंटेक्टे्स>>>>>>>>>>>>>>>>>>>
................
1
फेडिकल --- <फेडिकल>>>>>>>>>>>>>>>>>>>>>>>
................
2
सुरों --- <सर्सं>>>>>>>>>>>>>>>>>>>>>>>>
................
3
प्रदन्त --- <प्रदा्त>>>>>>>>>>>>>>>>>>>>>>
................
4
दीवारें --- <दीवारें>>>>>>>>>>>>>>>>>>>>>>
................
5
जलेगी --- <जलेगी>>>>>>>>>>>>>>>>>>>>>>>>
................
6
विल्वक --- <विलववक>>>>>>>>>>>>>>>>>>>>>>>
................
7
नाजुली --- <नजुुली>>>>>>>>>>>>>>>>>>>>>>>
................
8
अशोकधाम --- <अशोकधाम>>>>>>>>>>>>>>>>>>>>>>
................
9
माईसाहेब --- <मैसाहहेब>>>>>>>>>>>>>>>>>>>>>
................
10
नंदकिशोरभारतीय --- <नंदािषारिकर्तायय>>>>>>>>>>>>>
................
11
सम्राटाची --- <समरताताची>>>>>>>>>>>>>>>>>>>>
................
12
चन्नबसवानंद --- <चन्नानाववाद>>>>>>>>>>>>>>>>>>
................
13
पियाज्ज़ा --- <पियजज़ा>ा>>>>>>>>>>>>>>>>>>>>
................
14
फैब्रिका --- <फैब्रिका>>>>>>>>>>>>>>>>>>>>>
................
15
भावमूर्ति --- <भावमुर्ति>>>>>>>>

In [ ]:
word (self, source_batch,target_batch):
    max_len, batch_size = target_batch.shape
    outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)


            # last hidden & cell state of the encoder is used as the decoder's initial hidden state
    hidden, cell = self.encoder(source_batch)

    wordet=[]


    trg = torch.tensor(hindi_alphabet_to_index['<'])
    trg=trg.to(device)
    wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))
    for i in range(1, max_len):
        prediction, hidden, cell = self.decoder(trg, hidden, cell)
        outputs[i] = prediction
        trg = prediction.argmax(1)
        wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))


    return ''.join(wordet)

In [ ]:
word_from_vecs(X_valid[0],index_to_english_alphabet,False)

In [ ]:
word_from_vecs(y_valid[0],index_to_hindi_alphabet,False)

In [ ]:
next(iter(train_dataloader))

In [ ]:
s